In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Google-Stock-Prediction-V2'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,GOOG,2016-06-14 00:00:00+00:00,718.27,722.470,713.1200,716.48,1306065,718.27,722.470,713.1200,716.48,1306065,0.0,1.0
1,GOOG,2016-06-15 00:00:00+00:00,718.92,722.980,717.3100,719.00,1214517,718.92,722.980,717.3100,719.00,1214517,0.0,1.0
2,GOOG,2016-06-16 00:00:00+00:00,710.36,716.650,703.2600,714.91,1982471,710.36,716.650,703.2600,714.91,1982471,0.0,1.0
3,GOOG,2016-06-17 00:00:00+00:00,691.72,708.820,688.4515,708.65,3402357,691.72,708.820,688.4515,708.65,3402357,0.0,1.0
4,GOOG,2016-06-20 00:00:00+00:00,693.71,702.480,693.4100,698.77,2082538,693.71,702.480,693.4100,698.77,2082538,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,GOOG,2021-06-07 00:00:00+00:00,2466.09,2468.000,2441.0725,2451.32,1192453,2466.09,2468.000,2441.0725,2451.32,1192453,0.0,1.0
1254,GOOG,2021-06-08 00:00:00+00:00,2482.85,2494.495,2468.2400,2479.90,1253253,2482.85,2494.495,2468.2400,2479.90,1253253,0.0,1.0
1255,GOOG,2021-06-09 00:00:00+00:00,2491.40,2505.000,2487.3300,2499.50,1006337,2491.40,2505.000,2487.3300,2499.50,1006337,0.0,1.0
1256,GOOG,2021-06-10 00:00:00+00:00,2521.60,2523.260,2494.0000,2494.01,1561733,2521.60,2523.260,2494.0000,2494.01,1561733,0.0,1.0


In [4]:
data = data['high']

In [5]:
data = torch.from_numpy(np.array(data.tolist())).to(device).float().view(1,-1)

In [6]:
data_input = data[:,:-1]
data_target = data[:,1:]

In [7]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden  =512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.output = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [8]:
model = Model().to(device)
criterion = MSELoss()
optimizer = LBFGS(model.parameters(),lr=0.001)
epochs = 250

In [10]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(preds[n:])
        plt.plot(data_target.view(-1).cpu().detach().numpy())
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

Loss,1657026.5
_runtime,19
_timestamp,1634140101
_step,20
Val Loss,1657023.375


Loss,███▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
_runtime,▁▂▂▂▃▃▃▃▄▅▅▅▆▆▆▆▇▇▇██
_timestamp,▁▂▂▂▃▃▃▃▄▅▅▅▆▆▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
Val Loss,▁


wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  1%|▌                                          | 3/250 [00:42<58:39, 14.25s/it]


KeyboardInterrupt: 

In [11]:
torch.save(data,'./data.pt')
torch.save(data,'./data.pth')
torch.save(data_input,'data_input.pt')
torch.save(data_input,'data_input.pth')
torch.save(data_target,'data_target.pt')
torch.save(data_target,'data_target.pth')
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 185, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 749, in communicate_network_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 539, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 544, in _communicate_async
    